**Function for user input:**

- generates list of track URIs based on the mood specified
- returns random sample of URI links
- makes API call to spotify using list of URIs and creates playlist
- variables saved for use as playlist name

**Limitations:**

- only able to create playlist on my account (I do not have the ability currently to create playlists for other users based on my app permissions

In [1]:
import pandas as pd

import json
import requests
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
df = pd.read_csv('data/alt_moods_final.csv')

**Split into four separate dataframes based on mood label (not necessary but makes function less busy)**

In [3]:
calm = df[df['labels'] == 'calm']
happy = df[df['labels'] == 'happy']
energetic = df[df['labels'] == 'energetic']
sad = df[df['labels'] == 'sad']

In [4]:
def playlist():
    '''
    Takes no arguments.
    
    This function is used in order to generate a random sample of 12 song URIs in 
    1 of 4 mood-related dataframes (calm, happy, sad, energetic)
    
    User input: user enters a name, then choice of one of four moods, then a 
    title for the playlist. The title (playlist.title) is used as the playlist
    name in spotify. It is named thusly to reference outside of the function.
    
    '''
    playlist.name = input('Hi! What\'s your name? ')
    playlist.mood_choice = input("Select a mood - Calm, Energetic, Happy or Sad: ")
    playlist.title = input("What would you like to name your playlist? ")
    if playlist.mood_choice.lower() == 'calm':
        print('Run the cells below to retrieve your playlist!')
        return calm['uri'].sample(n=15)
    elif playlist.mood_choice.lower() == 'energetic':
        print('Run the cells below to retrieve your playlist!')
        return energetic['uri'].sample(n=15)
    elif playlist.mood_choice.lower() == 'happy':
        print('Run the cells below to retrieve your playlist!')
        return happy['uri'].sample(n=15)
    elif playlist.mood_choice.lower() == 'sad':
        print('Run the cells below to retrieve your playlist!')
        return sad['uri'].sample(n=15)
    else:
        return 'Mood not recognized. Please try again.'

**Run function for playlist name. Must cast as list to be readable when filling empty playlist with URIs (can not read pandas series).**

In [5]:
uris = list(playlist())

Hi! What's your name? Kelly
Select a mood - Calm, Energetic, Happy or Sad: calm
What would you like to name your playlist? Hi
Run the cells below to retrieve your playlist!


**Create empty playlist**

In [ ]:
#spotipy credentials
scope = 'playlist-modify-public'
user_id = 'SPOTIFY_USER_ID'
client_id = 'YOUR_CLIENT_ID'
client_secret = 'YOUR_CLIENT_SECRET'
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(user_id,scope,client_id=client_id,client_secret=client_secret,redirect_uri='http://localhost/8080') 
sp = spotipy.Spotify(auth=token)
playlist_name = playlist.title    
sp.user_playlist_create(user_id, name=playlist_name)

**Access Spotify playlist id to fill with songs generated in `playlist` function**

In [182]:
def playlist_id(user_id, playlist_name):
    playlist_id = ''
    playlists = sp.user_playlists(user_id)
    for playlist in playlists['items']:  # iterate through playlists in my account
        if playlist['name'] == playlist_name:  # filter for playlist I just created
            playlist_id = playlist['id']
    return playlist_id

**Fill playlist with songs!**

In [183]:
#unique playlist id
playlist_id = playlist_id(user_id, playlist_name)

#api endpoint
endpoint_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"

#retrieve songs with list of URIs
request_body = json.dumps({
          "uris" : uris
        })

#if response is 201, we're good to go
response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})

response

<Response [201]>

**Future things:**

- add additional parameters when classifying moods (ie thresholds for valence, energy, etc.)
- flask app to host playlist generator
- add genres to scraper